In [25]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import logging as log
log.basicConfig(level=log.DEBUG)

In [26]:
import sys
sys.path.append('../')
from baselines.utils import *
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

os.environ['WANDB_ENTITY'] = 'contract-nli-db'
os.environ['WANDB_PROJECT'] = 'contract-nli-metric'
os.environ['WANDB_LOG_MODEL'] = 'end'

In [27]:
from ipynb.fs.defs.contract_nli_bert_train import *

In [28]:
import torch

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [29]:
# cfg['model_name'] = 'nlpaueb/legal-bert-base-uncased'
cfg['model_name'] = 'bert-base-uncased'
cfg['trained_model_dir'] = '/scratch/shu7bh/contract_nli/trained_model/'
cfg['batch_size'] = 32
cfg

{'raw_data_dir': '../dataset/',
 'train_path': 'train.json',
 'test_path': 'test.json',
 'dev_path': 'dev.json',
 'model_name': 'bert-base-uncased',
 'max_length': 512,
 'models_save_dir': '/scratch/shu7bh/contract_nli/models',
 'dataset_dir': '/scratch/shu7bh/contract_nli/dataset',
 'results_dir': '/scratch/shu7bh/contract_nli/results',
 'trained_model_dir': '/scratch/shu7bh/contract_nli/trained_model/',
 'batch_size': 32}

In [30]:
# create dir if not exists
from pathlib import Path
Path(cfg["models_save_dir"]).mkdir(parents=True, exist_ok=True)
Path(cfg["dataset_dir"]).mkdir(parents=True, exist_ok=True)

In [31]:
tokenizer = AutoTokenizer.from_pretrained(cfg['model_name'])

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [32]:
from icecream import ic

In [33]:
train_data = load_data(os.path.join(cfg['raw_data_dir'], cfg['train_path']))
dev_data = load_data(os.path.join(cfg['raw_data_dir'], cfg['dev_path']))
test_data = load_data(os.path.join(cfg['raw_data_dir'], cfg['test_path']))

hypothesis = get_hypothesis(train_data)

train_data = train_data['documents']
dev_data = dev_data['documents']
test_data = test_data['documents']

train_data = train_data[:2]
dev_data = dev_data[:2]
test_data = test_data[:2]

ic.disable()

ic(len(train_data), len(dev_data), len(test_data))
train_dataset = NLIDataset(train_data, tokenizer, hypothesis, [1000, 1100, 1200, 1500], 50)
dev_dataset = NLIDataset(dev_data, tokenizer, hypothesis, [1000, 1100, 1200, 1500], 50)
test_dataset = NLIDataset(test_data, tokenizer, hypothesis, [1000, 1100, 1200, 1500], 50)

ic.enable()

del train_data
del dev_data
del test_data
del hypothesis

/home2/shu7bh/Courses/ANLP/Project/Contract-NLI/source_code/contract_nli_bert_train.ipynb:137: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "metadata": {},


In [34]:
print(len(train_dataset))
print(len(dev_dataset))
print(len(test_dataset))


1734
3026
2346


In [35]:
nli_weights, span_weight = get_class_weights(train_dataset)

In [36]:
nli_weights, span_weight

([0.8757575757575757, 0.7031630170316302, 2.2936507936507935],
 36.25806451612903)

In [37]:
from sklearn.metrics import precision_recall_curve
import numpy as np
def get_micro_average_precision_at_recall(y_true, y_pred, recall_level):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    return np.interp(recall_level, recall[::-1], precision[::-1])

In [38]:
# Import numpy and sklearn.metrics
import numpy as np
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
def calculate_micro_average_precision(y_true, y_pred):
    """Calculate the micro average precision score.

    Args:
        y_true (np.array): True labels.
        y_pred (np.array): Predicted labels.

    Returns:
        float: Micro average precision score.
    """
    # Get the number of classes
    num_classes = len(np.unique(y_true))
    
    if num_classes == 0:
        return 0.0

    # initialize the average precision score
    average_precision = 0.0

    # loop over all classes
    for class_idx in range(num_classes):
        # get the indices for this class
        y_true_indices = np.where(y_true == class_idx)
        # calculate the average precision score for this class
        average_precision += ic(precision_score(
            y_true[y_true_indices], y_pred[y_true_indices], average="micro"
        ))

    # return the average over all classes
    return average_precision / num_classes

In [39]:
from sklearn.metrics import f1_score
def calculate_f1_score_for_class(y_true, y_pred, class_idx):
    """Calculate the F1 score for a given class.

    Args:
        y_true (np.array): True labels.
        y_pred (np.array): Predicted labels.
        class_idx (int): Index of the class.

    Returns:
        float: F1 score for the given class.
    """
    # get the indices for the given class
    y_true_indices = np.where(y_true == class_idx)
    # calculate the F1 score for the given class
    return f1_score(
        y_true[y_true_indices], y_pred[y_true_indices], average="macro"
    )

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    auto_find_batch_size=True,
    output_dir=cfg['results_dir'],   # output directory
    num_train_epochs=10,            # total number of training epochs
    gradient_accumulation_steps=4,   # number of updates steps to accumulate before performing a backward/update pass
    logging_strategy='epoch',
    # eval_steps=0.25,
    # save_steps=0.25,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,
    label_names=['nli_label', 'span_labels', 'data_for_metrics'],
    report_to='none',
)

In [41]:
cfg['trained_model_dir']

'/scratch/shu7bh/contract_nli/trained_model/'

In [42]:
from transformers import EarlyStoppingCallback

In [43]:
import wandb
api = wandb.Api()
artifact = api.artifact('contract-nli-db/contract-nli-legal/model-swzofvm2:v0', type='model')
artifact_dir = artifact.download(cfg['trained_model_dir'])

model = ContractNLI.from_pretrained(artifact_dir).to(DEVICE)

DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home2/shu7bh/Courses/ANLP/Project/Contract-NLI/source_code, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home2/shu7bh/Courses/ANLP/Project/Contract-NLI/source_code, universal_newlines=False, shell=None, istream=None)
DEBUG:wandb.docker.auth:Trying paths: ['/home2/shu7bh/.docker/config.json', '/home2/shu7bh/.dockercfg']
DEBUG:wandb.docker.auth:No config file found
DEBUG:sentry_sdk.errors:[Tracing] Create new propagation context: {'trace_id': '62df3a1d0ca2457ebad056ae9ec7994f', 'span_id': '8256824dea33eeff', 'parent_span_id': None, 'dynamic_sampling_context': None}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 253
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 3

In [44]:
artifact_dir

'/scratch/shu7bh/contract_nli/trained_model/'

In [45]:
from transformers import Trainer
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from tqdm import tqdm
import numpy as np

class ContractNLIMetricTrainer(ContractNLITrainer):
    def __init__(self, *args, data_collator=None, **kwargs):
        super().__init__(*args, data_collator=data_collator, **kwargs)

    def evaluate(self, eval_dataset=None, ignore_keys=None):
        ic(eval_dataset)
        self.model.eval()
        self.dataloader = ic(self.get_eval_dataloader(eval_dataset))

        eval_span_labels = []
        eval_span_preds = []
        eval_nli_labels = []
        eval_nli_preds = []

        for inputs in tqdm(self.dataloader):
            inputs = self._prepare_inputs(inputs)
            ic(inputs['input_ids'].shape)
            span_labels = inputs.pop('span_labels')
            nli_labels = inputs.pop('nli_label')
            inputs.pop('data_for_metrics')

            with torch.no_grad():
                outputs = self.model(**inputs)
                span_logits, nli_logits = outputs[0], outputs[1]
                
                mask = span_labels != -1
                span_labels = span_labels[mask]
                span_logits = span_logits[mask]
                
                span_labels = span_labels.float()
                span_logits = span_logits.float()
                
                span_labels = span_labels.view(-1)
                span_logits = span_logits.view(-1)

                span_preds = ic(torch.sigmoid(span_logits)) >= 0.5
                # eval_span_preds = torch.tensor(eval_span_preds.squeeze(1), dtype=torch.long)
                nli_preds = torch.argmax(torch.softmax(nli_logits, dim=1), dim=1)

                eval_span_labels.extend(span_labels.cpu().numpy())
                eval_span_preds.extend(span_preds.cpu().numpy())
                eval_nli_labels.extend(nli_labels.cpu().numpy())
                eval_nli_preds.extend(nli_preds.cpu().numpy())


        eval_nli_acc = accuracy_score(eval_nli_labels, eval_nli_preds)

        ic.enable()
        ic(len(eval_span_labels), len(eval_span_preds))

        ic(sum(eval_span_labels), sum(eval_span_preds))

        mAP = calculate_micro_average_precision(torch.tensor(eval_span_labels), torch.tensor(eval_span_preds))        

        # mAP = average_precision_score(torch.tensor(true_span_labels), torch.tensor(pred_span_labels))
        precision_at_80_recall = get_micro_average_precision_at_recall(torch.tensor(eval_span_labels), torch.tensor(eval_span_preds), 0.8)
        f1_score_for_entailment = calculate_f1_score_for_class(torch.tensor(eval_nli_labels), torch.tensor(eval_nli_preds), get_labels()['Entailment'])
        f1_score_for_contradiction = calculate_f1_score_for_class(torch.tensor(eval_nli_labels), torch.tensor(eval_nli_preds), get_labels()['Contradiction'])
        
        return {
            'mAP' : mAP,
            'precision_at_80_recall' : precision_at_80_recall,
            'nli_acc': eval_nli_acc,
            'f1_score_for_entailment': f1_score_for_entailment,
            'f1_score_for_contradiction': f1_score_for_contradiction
        }

In [46]:
trainer = ContractNLIMetricTrainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=dev_dataset,            # evaluation dataset
    data_collator=ContractNLIMetricTrainer.collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.001)],
    model_init=model_init,
)

/home2/shu7bh/miniforge3/envs/nli/lib/python3.11/site-packages/transformers/trainer.py:366: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(


In [ ]:
ic.disable()
results = trainer.evaluate()
results